In [ ]:
# import general packages
import numpy as np
import pde

# import modules from this package
import phasesep as ph
print(ph.__file__)

In [ ]:
def run_simulation(
    d: float = 0.5, # diffusivity ratio
    chi: float = 4.0, # interaction parameter
    k: float = 0.1, # reaction rate
    h: float = 5, # reaction non-linearity
    dim: int = 1, # dimension of space
    shape: float = 100, # length of one axis in space
    size: float = 200, # number  of support points along each axis
):
    SEED = 8
    rng = np.random.default_rng(SEED)

    # set simulation time
    SIMULATION_TIME = 100000
    interval = float(SIMULATION_TIME / 100)

    # set grid
    if dim == 1:
        GRID = pde.CartesianGrid([[0, shape]], [size], periodic=True)
    if dim == 2:
        GRID = pde.CartesianGrid([[0, shape], [0, shape]], [size, size], periodic=True)

    # initial state
    PERTURBATION = pde.ScalarField.random_normal(grid=GRID, mean=0, std=2e-4, rng=rng)
    PHI = 0.2
    a = PHI + PERTURBATION
    b = PHI + PERTURBATION
    STATE = pde.FieldCollection([a, b])

    # set kappa matrix
    kappa = 1.0 * np.diag([1.0, 1.0, 0.0])

    # set chemical reactions
    H = h
    D = d
    hill = f"2*{k}*{PHI}*c[0]**{H}/(c[0]**{H}+c[1]**{H})"
    reactiona = ph.Reaction([1, 0, 0], f"{hill} - {k}*c[0]", conservative=False)
    reactionb = ph.Reaction([0, 1, 0], f"{hill} - {k}*c[1]", conservative=False)

    reactions = ph.Reactions(3, [reactiona, reactionb])

    # set interaction matrix
    CHI = chi
    chi = np.zeros((3, 3))
    chi[0, 1] = chi[1, 0] = CHI

    # define Florry Huggins free energy
    f = ph.FloryHugginsNComponents(3, chis=chi, variables=["a", "b"])

    # define Cahn Hilliard PDE including chemical reactions
    eq = ph.CahnHilliardMultiplePDE(
        {
            "free_energy": f,
            "kappa": kappa,
            "mobility": [1, D, 20],
            "reactions": reactions,
        }
    )

    # set trackers
    storage = pde.MemoryStorage()
    trackers = [
        "progress",
        storage.tracker(interval=interval),
        pde.PlotTracker(interval="0:10"),
        "steady_state",
    ]

    # run simulation
    sol = eq.solve(
        STATE,
        SIMULATION_TIME,
        method="explicit",
        tolerance=1e-6,
        adaptive=True,
        tracker=trackers,
    )

    # save movie
    if dim == 2:
        pde.movie(
            storage,
            f"./movie_2d_d{D}_chi{CHI}_k{k}_h{H}_dim{dim}_shape{shape}_size{size}.mp4",
            plot_args={"vmin": 0, "vmax": 1},
        )
    if dim == 1:
        pde.movie(
            storage,
            f"./movie_1d_d{D}_chi{CHI}_k{k}_h{H}_dim{dim}_shape{shape}_size{size}.mp4",
            plot_args={"ax_style": {"ylim": (0, 1)}},
        )

    # return final state
    solution = {
        "sol0": sol[0].data,
        "sol1": sol[1].data,
    }

    return solution

In [ ]:
##### 1d simulation #####
d = 9.6
chi = 0.0
k = 0.1
h = 5
dim = 1
shape = 100
size = 200
sol = run_simulation(d, chi, k, h, dim, shape, size)

In [ ]:
##### 2d simulation #####
d = 9.6
chi = -11
k = 0.01
h = 5
dim = 2
shape = 40
size = 60
sol = run_simulation(d, chi, k, h, dim, shape, size)